In [1]:
library(dplyr); library(readr); library(stringr)

dir.create("outputs/PhaseA_config", recursive = TRUE, showWarnings = FALSE)

# 1) Load the canonical item-level, ordered dataset (built earlier for Phase A prep)
items <- readRDS("outputs/PhaseA_prep/items_ordered_wide.rds")

# 2) Define item pools (DASS-21 and BRS) and labels for reporting
d_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S","dQ9A",
             "dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A","dQ16D","dQ17D","dQ18S","dQ19A","dQ20A","dQ21D")
r_items <- c("rQ1","rQ3","rQ5","rQ2_r","rQ4_r","rQ6_r")

pool_exist <- function(vs) intersect(vs, names(items))
dass_pool <- pool_exist(d_items)
brs_pool  <- pool_exist(r_items)

# 3) Verify ordinal levels for polychoric analyses (DASS 0-3, BRS 1-5)
chk_levels <- function(x) if (is.factor(x)) paste(levels(x), collapse = ",") else NA_character_
level_report <- data.frame(
  item = c(dass_pool, brs_pool),
  levels = sapply(items[, c(dass_pool, brs_pool), drop = FALSE], chk_levels)
)
write_csv(level_report, "outputs/PhaseA_config/levels_check.csv")

# 4) Create an item inventory to track decisions through A2–C
item_inventory <- tibble::tibble(
  item = c(dass_pool, brs_pool),
  domain = c(rep("DASS", length(dass_pool)), rep("BRS", length(brs_pool))),
  planned_factor = dplyr::case_when(
    str_detect(item, "D$") ~ "Dep",
    str_detect(item, "A$") ~ "Anx",
    str_detect(item, "S$") ~ "Str",
    domain == "BRS" ~ "BRS",
    TRUE ~ NA_character_
  ),
  keep_EFA = NA,       # fill after A3
  keep_ESEM = NA,      # fill after A4 (if run)
  reason = NA_character_,  # short note: low loading, cross-loading, content, DIF-risk
  notes = NA_character_
)
write_csv(item_inventory, "outputs/PhaseA_config/item_inventory.csv")

# 5) Set and save selection rules for transparency
rules <- list(
  factor_count_method = "Parallel analysis on polychoric (DASS, BRS separately)",
  rotation = "Oblimin (oblique)",
  factoring = "Principal axis (psych::fa, fm='pa')",
  primary_loading_min = 0.50,
  cross_loading_max = 0.30,
  communality_pref = 0.40,
  backup_loading_min = 0.45,   # soft buffer if needed for coverage
  target_items_per_factor = c(Dep = 2:4, Anx = 2:4, Str = 2:4, BRS = 2:3),
  references = c(
    "psych::fa.parallel; polychoric EFA best practice for ordinal items",
    "Primary >= .50, cross < .30 practical rules; oblique rotations recommended"
  )
)
saveRDS(rules, "outputs/PhaseA_config/selection_rules.rds")
writeLines(c(
  "Phase A1 configuration:",
  paste("Rotation:", rules$rotation),
  paste("Factoring:", rules$factoring),
  paste("Primary loading >=", rules$primary_loading_min, ", Cross-loading <", rules$cross_loading_max),
  paste("Communality preference >=", rules$communality_pref),
  paste("Target items per factor:", paste(names(rules$target_items_per_factor),
        sapply(rules$target_items_per_factor, function(x) paste(range(x), collapse='-')), collapse='; '))
), "outputs/PhaseA_config/selection_rules.txt")



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
library(psych); library(readr)

items <- readRDS("outputs/PhaseA_prep/items_ordered_wide.rds")
rules <- readRDS("outputs/PhaseA_config/selection_rules.rds")

d_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S","dQ9A",
             "dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A","dQ16D","dQ17D","dQ18S","dQ19A","dQ20A","dQ21D")
r_items <- c("rQ1","rQ3","rQ5","rQ2_r","rQ4_r","rQ6_r")

DASS <- items[, intersect(d_items, names(items)), drop = FALSE]
BRS  <- items[, intersect(r_items,  names(items)), drop = FALSE]

dir.create("outputs/PhaseA_A2", recursive = TRUE, showWarnings = FALSE)

# Parallel analysis plots (polychoric)
png("outputs/PhaseA_A2/fig_parallel_DASS.png", width = 1200, height = 800, res = 140)
fa.parallel(DASS, cor = "poly", fm = "pa", fa = "fa", n.iter = 500,
            main = "DASS: Parallel analysis (polychoric)")
dev.off()

png("outputs/PhaseA_A2/fig_parallel_BRS.png", width = 1200, height = 800, res = 140)
fa.parallel(BRS, cor = "poly", fm = "pa", fa = "fa", n.iter = 500,
            main = "BRS: Parallel analysis (polychoric)")
dev.off()

# Save eigenvalues for quick inspection
pc_DASS <- psych::polychoric(DASS)$rho
pc_BRS  <- psych::polychoric(BRS)$rho
ev_DASS <- eigen(pc_DASS, symmetric = TRUE)$values
ev_BRS  <- eigen(pc_BRS,  symmetric = TRUE)$values
readr::write_csv(data.frame(idx = seq_along(ev_DASS), eig = ev_DASS),
                 "outputs/PhaseA_A2/eigenvalues_DASS.csv")
readr::write_csv(data.frame(idx = seq_along(ev_BRS),  eig = ev_BRS),
                 "outputs/PhaseA_A2/eigenvalues_BRS.csv")


Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric input to numeric
Converted non-numeric inp

Converted non-numeric input to numeric


Converted non-numeric input to numeric


: [1m[33mError[39m:[22m
[33m![39m object 'pc_DASS' not found

In [ ]:
library(psych); library(dplyr); library(readr); library(tidyr)

# Load config and data
rules <- readRDS("outputs/PhaseA_config/selection_rules.rds")
primary_min <- rules$primary_loading_min
cross_max   <- rules$cross_loading_max
comm_pref   <- rules$communality_pref

items <- readRDS("outputs/PhaseA_prep/items_ordered_wide.rds")

d_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S","dQ9A",
             "dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A","dQ16D","dQ17D","dQ18S","dQ19A","dQ20A","dQ21D")
r_items <- c("rQ1","rQ3","rQ5","rQ2_r","rQ4_r","rQ6_r")

# Subsets
DASS_fac <- items[, intersect(d_items, names(items)), drop = FALSE]
BRS_fac  <- items[, intersect(r_items,  names(items)), drop = FALSE]

# Coerce to numeric matrices (keep ordinal meaning; polychoric will be computed inside fa)
fac_to_num <- function(df) {
  as.data.frame(lapply(df, function(x) {
    if (is.factor(x)) as.numeric(as.character(x)) else as.numeric(x)
  }))
}
DASS_num <- fac_to_num(DASS_fac)
BRS_num  <- fac_to_num(BRS_fac)

# Choose nfactors based on your parallel analysis results
nf_DASS <- 3
nf_BRS  <- 1

dir.create("outputs/PhaseA_A3", recursive = TRUE, showWarnings = FALSE)

# Run EFA: principal axis, oblimin, ask fa() to use polychoric correlations
efa_DASS <- psych::fa(DASS_num, nfactors = nf_DASS, fm = "pa", rotate = "oblimin", cor = "poly")
efa_BRS  <- psych::fa(BRS_num,  nfactors = nf_BRS,  fm = "pa", rotate = "oblimin", cor = "poly")

sink("outputs/PhaseA_A3/efa_DASS_summary.txt"); print(efa_DASS); sink()
sink("outputs/PhaseA_A3/efa_BRS_summary.txt");  print(efa_BRS);  sink()

# Tidy/flag loadings
tidy_loadings <- function(fa_obj, primary_min, cross_max, comm_pref){
  L <- as.data.frame(unclass(fa_obj$loadings))
  L$item <- rownames(fa_obj$loadings)
  L_long <- L %>%
    pivot_longer(cols = -item, names_to = "Factor", values_to = "loading") %>%
    group_by(item) %>%
    mutate(primary = abs(loading) == max(abs(loading), na.rm = TRUE)) %>%
    ungroup()
  prim <- L_long %>% filter(primary) %>% select(item, Factor, loading) %>%
    rename(primary_factor = Factor, primary_loading = loading)
  cross <- L_long %>% filter(!primary) %>%
    group_by(item) %>%
    summarize(max_cross = max(abs(loading), na.rm = TRUE), .groups = "drop")
  h2 <- data.frame(item = names(fa_obj$communality), communality = fa_obj$communality)
  out <- prim %>% left_join(cross, by = "item") %>% left_join(h2, by = "item") %>%
    mutate(
      pass_primary = abs(primary_loading) >= primary_min,
      pass_cross   = is.na(max_cross) | max_cross < cross_max,
      pass_comm    = is.na(communality) | communality >= comm_pref,
      keep_rule    = pass_primary & pass_cross & pass_comm
    )
  list(raw = L, long = L_long, summary = out)
}

dass_tidy <- tidy_loadings(efa_DASS, primary_min, cross_max, comm_pref)
brs_tidy  <- tidy_loadings(efa_BRS,  primary_min, cross_max, comm_pref)

write_csv(dass_tidy$raw %>% mutate(item = rownames(efa_DASS$loadings)),
          "outputs/PhaseA_A3/DASS_loadings_raw.csv")
write_csv(dass_tidy$summary, "outputs/PhaseA_A3/DASS_item_flags.csv")

write_csv(brs_tidy$raw %>% mutate(item = rownames(efa_BRS$loadings)),
          "outputs/PhaseA_A3/BRS_loadings_raw.csv")
write_csv(brs_tidy$summary, "outputs/PhaseA_A3/BRS_item_flags.csv")

# Sorted views for quick selection
dass_sorted <- dass_tidy$summary %>% arrange(primary_factor, desc(abs(primary_loading)))
brs_sorted  <- brs_tidy$summary  %>% arrange(primary_factor, desc(abs(primary_loading)))
write_csv(dass_sorted, "outputs/PhaseA_A3/DASS_loadings_sorted.csv")
write_csv(brs_sorted,  "outputs/PhaseA_A3/BRS_loadings_sorted.csv")


Warning message:
There was 1 warning in `summarize()`.
ℹ In argument: `max_cross = max(abs(loading), na.rm = TRUE)`.
Caused by warning in `max()`:
! no non-missing arguments to max; returning -Inf 


In [ ]:
library(dplyr); library(readr); library(stringr); library(tidyr)

rules <- readRDS("outputs/PhaseA_config/selection_rules.rds")
primary_min <- rules$primary_loading_min
cross_max   <- rules$cross_loading_max
comm_pref   <- rules$communality_pref
backup_min  <- rules$backup_loading_min

# Load tidy summaries created in A3
dass_flags <- readr::read_csv("outputs/PhaseA_A3/DASS_item_flags.csv", show_col_types = FALSE)
brs_flags  <- readr::read_csv("outputs/PhaseA_A3/BRS_item_flags.csv",  show_col_types = FALSE)

# Map DASS primary factor labels to domains (heuristic by item suffix)
map_domain <- function(item, primary_factor){
  # If available, use item suffix to domain; else keep primary_factor string
  if (str_detect(item, "D$")) return("Dep")
  if (str_detect(item, "A$")) return("Anx")
  if (str_detect(item, "S$")) return("Str")
  as.character(primary_factor)
}

dass_sel <- dass_flags %>%
  mutate(domain = mapply(map_domain, item, primary_factor),
         reason = NA_character_) %>%
  mutate(
    keep_EFA = case_when(
      keep_rule ~ TRUE,
      !keep_rule & (abs(primary_loading) >= backup_min) &
        (is.na(max_cross) | max_cross < cross_max) ~ NA,  # borderline, may keep for coverage
      TRUE ~ FALSE
    ),
    reason = case_when(
      keep_rule ~ "pass: primary/cross/comm",
      keep_EFA %in% NA ~ "borderline: loading >= backup_min",
      abs(primary_loading) < backup_min ~ "drop: low primary",
      !is.na(max_cross) & max_cross >= cross_max ~ "drop: cross-loading",
      !is.na(communality) & communality < comm_pref ~ "drop: low communality",
      TRUE ~ reason
    )
  )

brs_sel <- brs_flags %>%
  mutate(domain = "BRS",
         keep_EFA = ifelse(abs(primary_loading) >= backup_min, TRUE, FALSE),
         reason = ifelse(keep_EFA, "pass/borderline by loading", "drop: low primary"))

# Save annotated tables
dir.create("outputs/PhaseA_A3", showWarnings = FALSE)
readr::write_csv(dass_sel, "outputs/PhaseA_A3/DASS_item_flags_annotated.csv")
readr::write_csv(brs_sel,  "outputs/PhaseA_A3/BRS_item_flags_annotated.csv")

# 2–4 items per DASS domain + 1–2 alternates
pick_top <- function(df, domain, n_main = 3, n_alt = 2){
  sub <- df %>% filter(domain == domain) %>%
    arrange(desc(abs(primary_loading)))
  main <- sub %>% filter(keep_EFA %in% c(TRUE, NA)) %>% head(n_main)
  alt  <- sub %>% filter(!(item %in% main$item)) %>% head(n_alt)
  list(main = main, alt = alt)
}

domains <- c("Dep","Anx","Str")
shortlist_main <- bind_rows(lapply(domains, function(dom) pick_top(dass_sel, dom)$main))
shortlist_alt  <- bind_rows(lapply(domains, function(dom) pick_top(dass_sel, dom)$alt))

# BRS: pick 2–3 strongest (main) and 1–2 alternates
brs_main <- brs_sel %>% arrange(desc(abs(primary_loading))) %>% head(3)
brs_alt  <- brs_sel %>% arrange(desc(abs(primary_loading))) %>% slice(4:5)

readr::write_csv(shortlist_main, "outputs/PhaseA_A3/DASS_shortlist_main.csv")
readr::write_csv(shortlist_alt,  "outputs/PhaseA_A3/DASS_shortlist_alternates.csv")
readr::write_csv(brs_main,       "outputs/PhaseA_A3/BRS_shortlist_main.csv")
readr::write_csv(brs_alt,        "outputs/PhaseA_A3/BRS_shortlist_alternates.csv")

# Update item inventory with keep_EFA and reasons
inv_path <- "outputs/PhaseA_config/item_inventory.csv"
inv <- readr::read_csv(inv_path, show_col_types = FALSE)

mark_keep <- function(inv, sel){
  inv %>% left_join(sel %>% select(item, domain, keep_EFA, reason), by = c("item","domain")) %>%
    mutate(
      keep_EFA = coalesce(keep_EFA.y, keep_EFA.x),
      reason   = coalesce(reason.y, reason.x)
    ) %>%
    select(-keep_EFA.y, -keep_EFA.x, -reason.y, -reason.x)
}

inv2 <- inv %>% mark_keep(dass_sel) %>% mark_keep(brs_sel)
readr::write_csv(inv2, inv_path)


In [ ]:
library(dplyr); library(readr); library(stringr)

# Load annotated DASS and BRS selections
dass_sel <- readr::read_csv("outputs/PhaseA_A3/DASS_item_flags_annotated.csv", show_col_types = FALSE)
brs_sel  <- readr::read_csv("outputs/PhaseA_A3/BRS_item_flags_annotated.csv",  show_col_types = FALSE)

# Domain mapping from item suffix
dass_sel <- dass_sel %>%
  mutate(domain = case_when(
    str_detect(item, "D$") ~ "Dep",
    str_detect(item, "A$") ~ "Anx",
    str_detect(item, "S$") ~ "Str",
    TRUE ~ domain
  ))

# Function to take top items by absolute loading, de-duplicated
pick_clean <- function(df, domain, n_main = 3, n_alt = 2, min_loading = 0.5, max_cross = 0.3){
  sub <- df %>% filter(domain == domain) %>%
    arrange(desc(abs(primary_loading))) %>%
    distinct(item, .keep_all = TRUE)
  main <- sub %>%
    filter((abs(primary_loading) >= min_loading) & (is.na(max_cross) | max_cross < max_cross)) %>%
    head(n_main)
  # If fewer than n_main, top-up with next strongest candidates not already chosen
  if (nrow(main) < n_main) {
    filler <- sub %>% filter(!(item %in% main$item)) %>% head(n_main - nrow(main))
    main <- bind_rows(main, filler)
  }
  alt <- sub %>% filter(!(item %in% main$item)) %>% head(n_alt)
  list(main = main, alt = alt)
}

domains <- c("Dep","Anx","Str")
main_list <- lapply(domains, function(dom) pick_clean(dass_sel, dom)$main)
alt_list  <- lapply(domains, function(dom) pick_clean(dass_sel, dom)$alt)

dass_main <- bind_rows(main_list) %>% select(item, primary_factor, primary_loading, max_cross, communality, domain)
dass_alt  <- bind_rows(alt_list)  %>% select(item, primary_factor, primary_loading, max_cross, communality, domain)

# BRS: strongest 3 main, next 2 alternates, de-duplicated
brs_sorted <- brs_sel %>% arrange(desc(abs(primary_loading))) %>% distinct(item, .keep_all = TRUE)
brs_main   <- brs_sorted %>% head(3) %>% mutate(domain = "BRS")
brs_alt    <- brs_sorted %>% slice(4:5) %>% mutate(domain = "BRS")

dir.create("outputs/PhaseA_A3/final_shortlists", recursive = TRUE, showWarnings = FALSE)
readr::write_csv(dass_main, "outputs/PhaseA_A3/final_shortlists/DASS_shortlist_main.csv")
readr::write_csv(dass_alt,  "outputs/PhaseA_A3/final_shortlists/DASS_shortlist_alternates.csv")
readr::write_csv(brs_main,  "outputs/PhaseA_A3/final_shortlists/BRS_shortlist_main.csv")
readr::write_csv(brs_alt,   "outputs/PhaseA_A3/final_shortlists/BRS_shortlist_alternates.csv")


In [11]:
library(lavaan); library(dplyr); library(readr); library(stringr); library(tidyr)

# Load items and rules
items <- readRDS("outputs/PhaseA_prep/items_ordered_wide.rds")
rules <- readRDS("outputs/PhaseA_config/selection_rules.rds")
primary_min <- rules$primary_loading_min
cross_max   <- rules$cross_loading_max

# DASS items only
d_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S","dQ9A",
             "dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A","dQ16D","dQ17D","dQ18S","dQ19A","dQ20A","dQ21D")
DASS <- items[, intersect(d_items, names(items)), drop = FALSE]

# Target mapping by suffix: Dep =~ *D, Anx =~ *A, Str =~ *S
dep_items <- grep("D$", colnames(DASS), value = TRUE)
anx_items <- grep("A$", colnames(DASS), value = TRUE)
str_items <- grep("S$", colnames(DASS), value = TRUE)

# ESEM syntax: three EFA blocks with target rotation
model_esem <- '
 efa("block1")*Dep  =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
 efa("block2")*Anx  =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
 efa("block3")*Str  =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S
'

fit_esem <- lavaan::cfa(model_esem, data = DASS,
                        estimator = "WLSMV", parameterization = "theta", std.lv = TRUE,
                        ordered = colnames(DASS), rotation = "target")

# Extract rotated loadings
L <- parameterEstimates(fit_esem, standardized = TRUE) %>%
  filter(op == "=~") %>%
  select(lhs, rhs, est.std) %>%
  tidyr::pivot_wider(names_from = lhs, values_from = est.std) %>%
  rename(item = rhs)

# Long form and flags
L_long <- L %>% tidyr::pivot_longer(-item, names_to = "Factor", values_to = "loading")
prim <- L_long %>% group_by(item) %>% slice_max(order_by = abs(loading), n = 1, with_ties = FALSE) %>%
  ungroup() %>% rename(primary_factor = Factor, primary_loading = loading)
cross <- L_long %>% left_join(prim %>% select(item, primary_factor), by = "item") %>%
  filter(Factor != primary_factor) %>%
  group_by(item) %>% summarize(max_cross = max(abs(loading), na.rm = TRUE), .groups = "drop")

esem_flags <- prim %>% left_join(cross, by = "item") %>%
  mutate(
    pass_primary = abs(primary_loading) >= primary_min,
    pass_cross   = is.na(max_cross) | max_cross < cross_max,
    keep_ESEM    = pass_primary & pass_cross
  )

dir.create("outputs/PhaseA_A4", recursive = TRUE, showWarnings = FALSE)
readr::write_csv(L,           "outputs/PhaseA_A4/DASS_ESEM_loadings_wide.csv")
readr::write_csv(esem_flags,  "outputs/PhaseA_A4/DASS_ESEM_item_flags.csv")


This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.

Attaching package: ‘lavaan’

The following object is masked from ‘package:psych’:

    cor2cov



: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `est.std` doesn't exist.